In [0]:
# tech_list and datekey should be maintained
tech_list = [
    ('gsm', '2g', '900'),
    ('gsm', '2g', '1800'),
    ('umts', '3g', '900'),
    ('umts', '3g', '2100'),
    
    ('lte', '4g', '700'),
    ('lte', '4g', '800'),
    ('lte', '4g', '1800'),
    ('lte', '4g', '2100'),
    ('lte', '4g', '2600')
]
datekey = '2018-10-24'
year, month, day = datekey.split('-')


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, lit

spark = SparkSession.builder.appName("DimensionTableTransform").getOrCreate()



df_tech_dimension = spark.createDataFrame([
    (row[0], row[1], row[2], 'site_' + row[1] + '_cnt', 'frequency_band_' + row[0][0].upper() + row[2] ) for row in tech_list
], ["tech_type", "network_type", "frequency", "cell_cnt_value", "frequency_band_value"])
df_tech_dimension.show()

In [0]:
df_site = spark.read \
        .format("csv") \
        .option("header", "true") \
        .option("delimiter", ";") \
        .option("inferSchema", "true") \
        .load("gs://martech-archive-data/Archive/site/year={year}/month={mon}/day={day}/*.csv".format(year=year, mon=month, day=day))
    
df_site.show()



In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

# Define the schema
schema = StructType([
    StructField("year", IntegerType(), True),
    StructField("month", IntegerType(), True),
    StructField("day", IntegerType(), True),
    StructField("cell_identity", StringType(), True),
    StructField("frequency_band", StringType(), True),
    StructField("site_id", StringType(), True),
    StructField("tech_type", StringType(), True)
])

# Create an empty DataFrame with the defined schema
df_data = spark.createDataFrame(spark.sparkContext.emptyRDD(), schema)


# Merge all the data into a dataframe
tech_types = set(item[0] for item in tech_list)
for tech_type in tech_types:
    path = "gs://martech-archive-data/Archive/{type}/year={year}/month={mon}/day={day}/*.csv".format(type=tech_type, year=year, mon=month, day=day)
   
    try:
        df_tech_type = spark.read \
            .format("csv") \
            .option("header", "true") \
            .option("delimiter", ";") \
            .option("inferSchema", "true") \
            .load(path)
    except Exception as e:
        continue

    df_tech_type = df_tech_type.withColumn(tech_type, lit(tech_type))
    df_data = df_data.union(df_tech_type)
        

df_data.show()

In [0]:
df_data.createOrReplaceTempView("martech_archive_data")
df_site.createOrReplaceTempView("martech_archive_site")
df_tech_dimension.createOrReplaceTempView("dim_tech_info")

In [0]:
%sql
SELECT * FROM martech_archive_data;




In [0]:
%sql
-- site_technology_cnt
SELECT bb.year, bb.month, bb.day, bb.site_id,
       COALESCE(SUM(aa.site_2g_cnt), 0) AS site_2g_cnt, 
       COALESCE(SUM(aa.site_3g_cnt), 0) AS site_3g_cnt,
       COALESCE(SUM(aa.site_4g_cnt), 0) AS site_4g_cnt
FROM (
  SELECT *
  FROM (
      SELECT year, month, day, tech_type, site_id,
            COUNT(DISTINCT cell_identity) AS cell_cnt
      FROM martech_archive_data
      GROUP BY year, month, day, tech_type, site_id
  )  a 
  JOIN (
      SELECT tech_type, cell_cnt_value
        FROM dim_tech_info
       GROUP BY tech_type, cell_cnt_value
  )  b ON a.tech_type = b.tech_type
  PIVOT (
    SUM(cell_cnt) AS cell_cnt
    FOR cell_cnt_value IN ('site_2g_cnt', 'site_3g_cnt', 'site_4g_cnt') 
  )
) aa RIGHT JOIN martech_archive_site bb ON aa.site_id = bb.site_id
GROUP BY bb.year, bb.month, bb.day, bb.site_id
;

In [0]:
%sql

-- frequency_band 
SELECT bb.year, bb.month, bb.day, bb.site_id,
       -- frequency_band
       MAX(COALESCE(frequency_band_G900, 0)) AS frequency_band_G900,
       MAX(COALESCE(frequency_band_G1800, 0)) AS frequency_band_G1800,
       MAX(COALESCE(frequency_band_U900, 0)) AS frequency_band_U900,
       MAX(COALESCE(frequency_band_U2100, 0)) AS frequency_band_U2100,
       -- frequency_band: lte
       MAX(COALESCE(frequency_band_L700, 0)) AS frequency_band_L700,
       MAX(COALESCE(frequency_band_L800, 0)) AS frequency_band_L800,
       MAX(COALESCE(frequency_band_L1800, 0)) AS frequency_band_L1800,
       MAX(COALESCE(frequency_band_L2100, 0)) AS frequency_band_L2100,
       MAX(COALESCE(frequency_band_L2600, 0)) AS frequency_band_L2600
FROM (
  SELECT *
    FROM (
        SELECT year, month, day, tech_type, site_id, frequency_band
        FROM martech_archive_data
    )  a 
    JOIN (
        SELECT tech_type, frequency, frequency_band_value
          FROM dim_tech_info
        GROUP BY tech_type, frequency, frequency_band_value
    )  b ON a.tech_type = b.tech_type AND a.frequency_band = b.frequency
    PIVOT (
      MAX(IF(frequency_band IS NOT NULL, 1, 0)) AS frenquency_is_exist
      FOR frequency_band_value IN ('frequency_band_G900', 'frequency_band_G1800', 'frequency_band_U900', 'frequency_band_U2100',
      'frequency_band_L700', 'frequency_band_L800', 'frequency_band_L1800', 'frequency_band_L2100', 'frequency_band_L2600') 
    )
) aa RIGHT JOIN martech_archive_site bb ON aa.site_id = bb.site_id
GROUP BY bb.year, bb.month, bb.day, bb.site_id
;

In [0]:
%sql
SELECT  year, month, day, site_id,
        SUM(site_2g_cnt) AS site_2g_cnt, 
        SUM(site_3g_cnt) AS site_3g_cnt,
        SUM(site_4g_cnt) AS site_4g_cnt,
        SUM(frequency_band_G900) AS frequency_band_G900,
        SUM(frequency_band_G1800) AS frequency_band_G1800,
        SUM(frequency_band_U900) AS frequency_band_U900,
        SUM(frequency_band_U2100) AS frequency_band_U2100,
        SUM(frequency_band_L700) AS frequency_band_L700,
        SUM(frequency_band_L800) AS frequency_band_L800,
        SUM(frequency_band_L1800) AS frequency_band_L1800,
        SUM(frequency_band_L2100) AS frequency_band_L2100,
        SUM(frequency_band_L2600) AS frequency_band_L2600
FROM (
  -- site_technology_cnt
  SELECT bb.year, bb.month, bb.day, bb.site_id,
        COALESCE(SUM(aa.site_2g_cnt), 0) AS site_2g_cnt, 
        COALESCE(SUM(aa.site_3g_cnt), 0) AS site_3g_cnt,
        COALESCE(SUM(aa.site_4g_cnt), 0) AS site_4g_cnt,
        0 AS frequency_band_G900,
        0 AS frequency_band_G1800,
        0 AS frequency_band_U900,
        0 AS frequency_band_U2100,
        0 AS frequency_band_L700,
        0 AS frequency_band_L800,
        0 AS frequency_band_L1800,
        0 AS frequency_band_L2100,
        0 AS frequency_band_L2600
  FROM (
    SELECT *
    FROM (
        SELECT year, month, day, tech_type, site_id,
              COUNT(DISTINCT cell_identity) AS cell_cnt
        FROM martech_archive_data
        GROUP BY year, month, day, tech_type, site_id
    )  a 
    JOIN (
        SELECT tech_type, cell_cnt_value
          FROM dim_tech_info
        GROUP BY tech_type, cell_cnt_value
    )  b ON a.tech_type = b.tech_type
    PIVOT (
      SUM(cell_cnt) AS cell_cnt
      FOR cell_cnt_value IN ('site_2g_cnt', 'site_3g_cnt', 'site_4g_cnt') 
    )
  ) aa RIGHT JOIN martech_archive_site bb ON aa.site_id = bb.site_id
  GROUP BY bb.year, bb.month, bb.day, bb.site_id

  UNION ALL 
  -- frequency_band 
  SELECT bb.year, bb.month, bb.day, bb.site_id,
        0 AS site_2g_cnt, 
        0 AS site_3g_cnt,
        0 AS site_4g_cnt,
        -- frequency_band
        MAX(COALESCE(frequency_band_G900, 0)) AS frequency_band_G900,
        MAX(COALESCE(frequency_band_G1800, 0)) AS frequency_band_G1800,
        MAX(COALESCE(frequency_band_U900, 0)) AS frequency_band_U900,
        MAX(COALESCE(frequency_band_U2100, 0)) AS frequency_band_U2100,
        -- frequency_band: lte
        MAX(COALESCE(frequency_band_L700, 0)) AS frequency_band_L700,
        MAX(COALESCE(frequency_band_L800, 0)) AS frequency_band_L800,
        MAX(COALESCE(frequency_band_L1800, 0)) AS frequency_band_L1800,
        MAX(COALESCE(frequency_band_L2100, 0)) AS frequency_band_L2100,
        MAX(COALESCE(frequency_band_L2600, 0)) AS frequency_band_L2600
  FROM (
    SELECT *
      FROM (
          SELECT year, month, day, tech_type, site_id, frequency_band
          FROM martech_archive_data
      )  a 
      JOIN (
          SELECT tech_type, frequency, frequency_band_value
            FROM dim_tech_info
          GROUP BY tech_type, frequency, frequency_band_value
      )  b ON a.tech_type = b.tech_type AND a.frequency_band = b.frequency
      PIVOT (
        MAX(IF(frequency_band IS NOT NULL, 1, 0)) AS frenquency_is_exist
        FOR frequency_band_value IN ('frequency_band_G900', 'frequency_band_G1800', 'frequency_band_U900', 'frequency_band_U2100',
        'frequency_band_L700', 'frequency_band_L800', 'frequency_band_L1800', 'frequency_band_L2100', 'frequency_band_L2600') 
      )
  ) aa RIGHT JOIN martech_archive_site bb ON aa.site_id = bb.site_id
  GROUP BY bb.year, bb.month, bb.day, bb.site_id
) aaa GROUP BY year, month, day, site_id

In [0]:
%sql

SELECT bb.year, bb.month, bb.day, bb.site_id,
       COALESCE(SUM(aa.site_2g_cnt), 0) AS site_2g_cnt,
       COALESCE(SUM(aa.site_3g_cnt), 0) AS site_3g_cnt,
       COALESCE(SUM(aa.site_4g_cnt), 0) AS site_4g_cnt
FROM (
  SELECT *
  FROM (
      SELECT year, month, day, tech_type, site_id,
            COUNT(DISTINCT cell_identity) AS cell_cnt
      FROM martech_archive_data
      GROUP BY year, month, day, tech_type, site_id
  )  a 
  JOIN (
      SELECT tech_type, cell_cnt_value
        FROM dim_tech_info
       GROUP BY tech_type, cell_cnt_value
  )  b ON a.tech_type = b.tech_type
  PIVOT (
    SUM(cell_cnt) AS cell_cnt
    FOR cell_cnt_value IN ('site_2g_cnt', 'site_3g_cnt', 'site_4g_cnt') 
  )
) aa RIGHT JOIN martech_archive_site bb ON aa.site_id = bb.site_id
GROUP BY bb.year, bb.month, bb.day, bb.site_id


In [0]:
%sql

-- frequency_band 
SELECT bb.year, bb.month, bb.day, bb.site_id,
       -- frequency_band
       MAX(COALESCE(frequency_band_G900, 0)) AS frequency_band_G900,
       MAX(COALESCE(frequency_band_U900, 0)) AS frequency_band_U900,
       MAX(COALESCE(frequency_band_G1800, 0)) AS frequency_band_G1800,
       MAX(COALESCE(frequency_band_U2100, 0)) AS frequency_band_U2100,
       MAX(COALESCE(frequency_band_L2600, 0)) AS frequency_band_L2600,
       MAX(COALESCE(frequency_band_L2100, 0)) AS frequency_band_L2100,
       MAX(COALESCE(frequency_band_L700, 0)) AS frequency_band_L700,
       MAX(COALESCE(frequency_band_L1800, 0)) AS frequency_band_L1800,
       MAX(COALESCE(frequency_band_L800, 0)) AS frequency_band_L800
FROM (
  SELECT *
    FROM (
        SELECT year, month, day, tech_type, site_id, frequency_band
        FROM martech_archive_data
    )  a 
    JOIN (
        SELECT tech_type, frequency, frequency_band_value
          FROM dim_tech_info
        GROUP BY tech_type, frequency, frequency_band_value
    )  b ON a.tech_type = b.tech_type AND a.frequency_band = b.frequency
    PIVOT (
      MAX(IF(frequency_band IS NOT NULL, 1, 0)) AS frenquency_is_exist
      FOR frequency_band_value IN ('frequency_band_G900', 'frequency_band_U900', 'frequency_band_G1800', 'frequency_band_U2100', 'frequency_band_L2600', 'frequency_band_L2100', 'frequency_band_L700', 'frequency_band_L1800', 'frequency_band_L800') 
    )
) aa RIGHT JOIN martech_archive_site bb ON aa.site_id = bb.site_id
GROUP BY bb.year, bb.month, bb.day, bb.site_id
